# Escherichia coli and Saccharomices cerevisiae co-culture

The notebook illustrates how to 
- construct a community model representing the co-culture of Escherichia coli and Saccharomices cerevisiae from models of each single organism,
- run FBA on the community model
- optimize the co-culture for the production of a naringenin.

In [1]:
from cobra.io import read_sbml_model

from mewpy.optimization import EA
from mewpy.optimization.evaluation import TargetFlux, BPCY
from mewpy.problems import RKOProblem
from mewpy import get_simulator
from mewpy.com import *

2026-01-03 16:15:41,224	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.8.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## Load individual organism model

In [2]:
sc = read_sbml_model('models/yeast/iMM904.xml.gz')
ec = read_sbml_model('models/ec/iAF1260.xml')
get_simulator(ec).objective='BIOMASS'

No objective coefficients in model. Unclear what should be optimized


# Community Model and Medium

In [3]:
community = CommunityModel([ec,sc])
cmodel = community.merged_model

Organism: 100%|██████████████████████████████████████████████████████████████| 2/2 [00:28<00:00, 14.19s/it]


In [4]:
for rx in cmodel.get_exchange_reactions():
    cmodel.set_reaction_bounds(rx,0,1000,False)

In [5]:
medium = dict()

minimal_medium = ['R_EX_xyl__D_e', 'R_EX_ac_e_iMM904', 'R_EX_mobd_e', 'R_EX_cl_e', 
                  'R_EX_cobalt2_e','R_EX_mn2_e', 'R_EX_mg2_e', 'R_EX_ca2_e', 'R_EX_k_e', 
                  'R_EX_zn2_e', 'R_EX_xyl__D_e_iAF1260', 'R_EX_so4_e', 
                  'R_EX_cu_e', 'R_EX_fe3_e', 'R_EX_o2_e', 'R_EX_pi_e', 
                  'R_EX_nh4_e']

# Carbon sources are constrained
for exchange in minimal_medium:
    if exchange != 'R_EX_xyl__D_e' and exchange != 'R_EX_ac_e_iMM904':
        medium[exchange]=(-1000, 0)
    else:
        medium[exchange]=(-10, 1000)

# Guarantee that L-Tyrosine is only produced by the organisms
medium['R_EX_tyr__L_e'] = 0

cmodel.set_environmental_conditions(medium)

In [6]:
res = cmodel.simulate(method='pFBA')

CplexSolverError: CPLEX Error  1016: Community Edition. Problem size limits exceeded. Purchase at http://ibm.biz/error1016.

In [ ]:
res.get_metabolite('M_tyr__L_e')

In [ ]:
cmodel.FVA(reactions=['R_EX_tyr__L_e_iAF1260','R_EX_tyr__L_e_iMM904'])

## Optimization

In [ ]:
TARGET = 'R_EX_tyr__L_e_iAF1260'

In [ ]:
BIOMASS = list(cmodel.objective.keys())[0]
BIOMASS

In [ ]:
f1 = BPCY(BIOMASS,TARGET)
f2 = TargetFlux(TARGET)

In [ ]:
essential = cmodel.essential_reactions()

In [ ]:
KO_targets = []

for rxn in cmodel.reactions:
        if rxn.endswith('iAF1260') and rxn not in essential:
            if (rxn == 'R_ATPM_iAF1260'
                or rxn.startswith('R_EX_') 
                or rxn.startswith('R_ATPS')
                or rxn.endswith('tex_iAF1260')
                or rxn.endswith('pp_iAF1260')
                or rxn.endswith('exi_iAF1260')):
                continue
            else:
                KO_targets.append(rxn)

In [ ]:
len(KO_targets)

In [ ]:
problem = RKOProblem(cmodel, 
                     fevaluation=[f1,f2],
                     target=KO_targets,
                     candidate_max_size=2)

ea = EA(problem, max_generations = 10)

In [ ]:
ea.run(simplify=False)

In [ ]:
ea.dataframe()

# Evaluate solutions

In [ ]:
solution = {'R_PYK_iAF1260':0, 'R_PPNDH_iAF1260':0}
res = cmodel.simulate(method='pFBA',constraints=solution)

In [ ]:
res.find('tyr')

In [ ]:
res.find('BIO')

Identify additional interactions:

In [ ]:
exchanges(community,res)